# Desafio Rox Patner

In [ ]:
%pip install awswrangler
%pip install boto3
%pip install dotenv
%pip install pyathena

In [1]:

import warnings 
import boto3
import awswrangler as wr
import pandas as pd
from pyathena import connect
import os
import dotenv



dotenv.load_dotenv()
warnings.filterwarnings('ignore')


# conexão com a AWS
session = boto3.Session(
    aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
    region_name=os.environ['AWS_REGION']
)

s3_client = session.client('s3')

c:\Users\marqu\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
#Ler os arquivos csv
person_person = pd.read_csv('data\Person.Person.csv', sep = ';', decimal=",")
production_product = pd.read_csv('data\Production.Product.csv', sep = ';', decimal=",")
sales_customer = pd.read_csv('data\Sales.Customer.csv', sep = ';', decimal=",")
sales_orderdetail = pd.read_csv('data\Sales.SalesOrderDetail.csv', sep = ';')
sales_orderheader = pd.read_csv('data\Sales.SalesOrderHeader.csv', sep = ';', decimal=",")
sales_specialofferproduct = pd.read_csv('data\Sales.SpecialOfferProduct.csv', sep = ';', decimal=",")

#Definindo os nomes nos schemas e das tabelas
dataframe_names = ["person_person","production_product", "sales_customer", "sales_orderdetail", "sales_orderheader", "sales_specialofferproduct"]
dataframes = [person_person, production_product, sales_customer, sales_orderdetail, sales_orderheader, sales_specialofferproduct]


In [3]:
#Processo de inserir os dados no S3 e criar as tabelas dentro do Athena

for dataframe,name_file in zip(dataframes,dataframe_names):

    wr.s3.to_parquet(
        df=dataframe,
        path=f"s3://{os.environ['RAW']}/{name_file}", # caminho do bucket 
        dataset=True,
        database=os.environ['RAW'], # Nome do database Athena
        mode = "overwrite", # caso já tenha processado o arquivo ele sobrescreve
        table=f"{name_file}",
        boto3_session = session 
    )

    print(f"O arquivo: {name_file} foi inserido no bucket com sucesso")

O arquivo: person_person foi inserido no bucket com sucesso
O arquivo: production_product foi inserido no bucket com sucesso
O arquivo: sales_customer foi inserido no bucket com sucesso
O arquivo: sales_orderdetail foi inserido no bucket com sucesso
O arquivo: sales_orderheader foi inserido no bucket com sucesso
O arquivo: sales_specialofferproduct foi inserido no bucket com sucesso


In [4]:
#Conector de acesso as tabelas no athena para realizar as consultas SQL

def connect_aws():
  
  conn = connect(aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
                aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
                s3_staging_dir=f"s3://{os.environ['S3_STANGING_DIR']}/",  # bucket em que as consultas devem ser salvas 
                region_name=os.environ['AWS_REGION'])
 
  return conn
teste = connect_aws()

# Desenvolvendo as questões do desafio


1.	Escreva uma query que retorna a quantidade de linhas na tabela Sales.SalesOrderDetail pelo campo SalesOrderID, desde que tenham pelo menos três linhas de detalhes.

2.	Escreva uma query que ligue as tabelas Sales.SalesOrderDetail, Sales.SpecialOfferProduct e Production.Product e retorne os 3 produtos (Name) mais vendidos (pela soma de OrderQty), agrupados pelo número de dias para manufatura (DaysToManufacture).

3.	Escreva uma query ligando as tabelas Person.Person, Sales.Customer e Sales.SalesOrderHeader de forma a obter uma lista de nomes de clientes e uma contagem de pedidos efetuados.

4.	Escreva uma query usando as tabelas Sales.SalesOrderHeader, Sales.SalesOrderDetail e Production.Product, de forma a obter a soma total de produtos (OrderQty) por ProductID e OrderDate.

5.	Escreva uma query mostrando os campos SalesOrderID, OrderDate e TotalDue da tabela Sales.SalesOrderHeader. Obtenha apenas as linhas onde a ordem tenha sido feita durante o mês de setembro/2011 e o total devido esteja acima de 1.000. Ordene pelo total devido decrescente.


In [5]:
# 1.

registros_com_mais_3l_na_Tabela = pd.read_sql(
"""
SELECT SalesOrderID, COUNT(*) AS NumDeRegDet
FROM "pipelineroxpatner".sales_orderdetail
GROUP BY SalesOrderID
HAVING COUNT(*) >= 3
""",
connect_aws())

print(f"Total de Linhas: {registros_com_mais_3l_na_Tabela['NumDeRegDet'].sum()}\n")


Total de Linhas: 94157



In [6]:
# 2.

top3_produtos_ordenados_OrderQty = pd.read_sql(
"""
SELECT
    pp.Name AS ProductName,
    pp.DaysToManufacture,
    SUM(sod.OrderQty) AS TotalOrderQuantity
FROM
"pipelineroxpatner".sales_orderdetail sod
INNER JOIN "pipelineroxpatner".sales_specialofferproduct ssop
  ON sod.specialofferid = ssop.specialofferid and sod.productid = ssop.productid
INNER JOIN "pipelineroxpatner".production_product pp
  ON pp.productid = ssop.productid
GROUP BY
    pp.Name, pp.DaysToManufacture
ORDER BY
    SUM(sod.OrderQty) DESC
LIMIT 3
""",
connect_aws()
)

print(f"Principais produtos mais vendidos:\n {top3_produtos_ordenados_OrderQty}")

Principais produtos mais vendidos:
               ProductName  DaysToManufacture  TotalOrderQuantity
0            AWC Logo Cap                  0                8311
1   Water Bottle - 30 oz.                  0                6815
2  Sport-100 Helmet, Blue                  0                6743


In [7]:
# 3.

cliente_por_total_pedido_efetuado = pd.read_sql(
"""
SELECT
    pe.businessentityid as PersonID,
    CONCAT(coalesce(pe.FirstName,''), ' ', COALESCE(pe.middlename,''), ' ', COALESCE(pe.LastName,'')) AS CustomerName,
    COUNT(soh.SalesOrderID) AS NumberOfOrders
FROM
    "pipelineroxpatner".Person_Person pe
INNER JOIN  "pipelineroxpatner".sales_customer sc 
    ON pe.BusinessEntityID = sc.PersonID
INNER JOIN "pipelineroxpatner".sales_orderheader soh
    ON sc.CustomerID = soh.CustomerID
GROUP BY
    1,2
ORDER BY
    COUNT(soh.SalesOrderID) desc
""",
connect_aws()
)

print(f"Pedidos por cliente: \n {cliente_por_total_pedido_efetuado.head(10)}")

Pedidos por cliente: 
    PersonID        CustomerName  NumberOfOrders
0      4515       Dalton  Perez              28
1     15994     Mason D Roberts              28
2     15978      Henry B Garcia              27
3      5409     Jason L Griffin              27
4     12569   Ashley  Henderson              27
5     15449     Nancy E Chapman              27
6     18668       Daniel  Davis              27
7     12526   Samantha  Jenkins              27
8      4855   Charles P Jackson              27
9      3197  Hailey I Patterson              27


In [8]:
#4.

produto_agrupado_por_data_e_por_OrderQty = pd.read_sql(
"""SELECT
        sod.ProductID,
        pp.name ProductName,
        soh.OrderDate,
        SUM(sod.OrderQty) AS TotalOrderQuantity
    FROM
        "pipelineroxpatner".sales_orderheader soh
    INNER JOIN "pipelineroxpatner".sales_orderdetail sod
        ON sod.salesorderid = soh.salesorderid
    INNER JOIN "pipelineroxpatner".production_product pp
        ON pp.productid = sod.productid
    GROUP BY
        sod.ProductID, pp.name, soh.OrderDate
""",
connect_aws()
)

print(f"Pedidos por Produto e Data: \n {produto_agrupado_por_data_e_por_OrderQty.head(5)}")

Pedidos por Produto e Data: 
    ProductID                     ProductName                OrderDate  \
0        711          Sport-100 Helmet, Blue  2011-05-31 00:00:00.000   
1        714      Long-Sleeve Logo Jersey, M  2011-05-31 00:00:00.000   
2        778          Mountain-100 Black, 48  2011-05-31 00:00:00.000   
3        742  HL Mountain Frame - Silver, 46  2011-05-31 00:00:00.000   
4        715      Long-Sleeve Logo Jersey, L  2011-05-31 00:00:00.000   

   TotalOrderQuantity  
0                  33  
1                  16  
2                  20  
3                   3  
4                  49  


In [9]:
# 5.

registros_filtrados_por_data_e_por_TotalDue = pd.read_sql(
"""
SELECT
    SalesOrderID,
    OrderDate,
    TotalDue
FROM
    "pipelineroxpatner".sales_orderheader
WHERE
    YEAR(CAST(OrderDate as TIMESTAMP)) = 2011
    AND MONTH(CAST(OrderDate AS TIMESTAMP)) = 9
    AND TotalDue > 1000
ORDER BY
    TotalDue DESC;
""",
connect_aws()
)

print(f"Pedidos por Produto e Data: \n {registros_filtrados_por_data_e_por_TotalDue.head()}")

Pedidos por Produto e Data: 
    SalesOrderID                OrderDate   TotalDue
0         44324  2011-09-01 00:00:00.000  3953.9884
1         44441  2011-09-22 00:00:00.000  3953.9884
2         44443  2011-09-22 00:00:00.000  3953.9884
3         44444  2011-09-24 00:00:00.000  3953.9884
4         44445  2011-09-25 00:00:00.000  3953.9884
